## Arquitectura del Sistema

### Base de Datos Vectorial

El sistema utiliza **PostgreSQL con pgvector** para almacenar y buscar embeddings:

```
┌─────────────────────────────────────────┐
│      Tabla: rag_embeddings              │
├─────────────────────────────────────────┤
│ id              (PK)                    │
│ text            (Texto original)        │
│ group_name      (Categoría)            │
│ intent          (Intención específica)  │
│ meta            (Metadata JSON)         │
│ embedding       (Vector 768 dims)       │
└─────────────────────────────────────────┘
```

##  Modelo de Datos - RAGEmbedding

La clase `RAGEmbedding` es el modelo ORM que representa la tabla de embeddings:

In [ ]:
from sqlalchemy import Column, Integer, Text, String
from sqlalchemy.dialects.postgresql import JSONB
from pgvector.sqlalchemy import Vector

class RAGEmbedding(Base):
    """
    Modelo para embeddings del sistema RAG
    
    Campos:
    - text: Texto completo (pregunta + respuesta)
    - group_name: Grupo de clasificación (ej: 'faq_empresa')
    - intent: Intención específica (ej: 'faq_manual')
    - meta: Información adicional en formato JSON
    - embedding: Vector de 768 dimensiones generado por Ollama
    """
    __tablename__ = "rag_embeddings"

    id = Column(Integer, primary_key=True, autoincrement=True)
    text = Column(Text, nullable=False)
    group_name = Column(String(100), nullable=False, index=True)
    intent = Column(String(150), nullable=False, index=True)
    meta = Column(JSONB, nullable=True)
    embedding = Column(Vector(768), nullable=False)

##  Inicialización del Sistema

Antes de usar el sistema RAG, debemos crear la extensión pgvector y las tablas necesarias:

In [ ]:
from sqlalchemy import create_engine, text

def create_vector_tables():
    """
    Inicializa la base de datos para RAG:
    1. Crea la extensión pgvector en PostgreSQL
    2. Crea la tabla rag_embeddings
    """
    with engine.connect() as conn:
        # Habilitar pgvector
        conn.execute(text("CREATE EXTENSION IF NOT EXISTS vector;"))
        conn.commit()

    # Crear tablas definidas en Base
    Base.metadata.create_all(engine)
    print("✓ Extensión vector verificada y tablas creadas")

# Ejemplo de uso
# create_vector_tables()

##  Inserción de Embeddings

### Inserción Individual

Para agregar un solo embedding:

In [ ]:
def add_embedding(
    text: str,
    group_name: str,
    intent: str,
    embedding: list[float],
    meta: dict = None
) -> int:
    """
    Inserta un embedding en la base de datos
    
    Parámetros:
    - text: Texto completo a almacenar
    - group_name: Categoría del contenido
    - intent: Intención específica
    - embedding: Vector generado (768 dimensiones)
    - meta: Metadata adicional (opcional)
    
    Retorna: ID del registro creado
    """
    session = get_db_session()
    
    try:
        rag = RAGEmbedding(
            text=text,
            group_name=group_name,
            intent=intent,
            meta=meta or {},
            embedding=embedding
        )
        session.add(rag)
        session.commit()
        session.refresh(rag)
        return rag.id
    except Exception as e:
        session.rollback()
        raise
    finally:
        session.close()

# Ejemplo de uso
# vector = embedder.embed_query("¿Cuál es el horario de atención?")
# add_embedding(
#     text="Pregunta: ¿Horarios? Respuesta: 9am-6pm lun-vie",
#     group_name="faq_empresa",
#     intent="faq_manual",
#     embedding=vector,
#     meta={"tipo": "horarios"}
# )

### Inserción por Lotes

Para optimizar la carga masiva de datos:

In [ ]:
def add_batch_embeddings(items: list[dict]) -> int:
    """
    Inserta múltiples embeddings en una sola transacción
    
    Cada item debe tener:
    {
        "text": str,
        "group_name": str,
        "intent": str,
        "embedding": list[float],
        "meta": dict (opcional)
    }
    
    Retorna: Cantidad de registros insertados
    """
    session = get_db_session()
    
    try:
        objs = [
            RAGEmbedding(
                text=it["text"],
                group_name=it["group_name"],
                intent=it["intent"],
                embedding=it["embedding"],
                meta=it.get("meta", {})
            )
            for it in items
        ]
        
        session.add_all(objs)
        session.commit()
        return len(objs)
    except Exception as e:
        session.rollback()
        raise
    finally:
        session.close()

##  Búsqueda Semántica

La función más importante del sistema: buscar información relevante basándose en similitud vectorial.

### Distancia Coseno

Utilizamos **distancia coseno** para medir similitud:
- **0** = Vectores idénticos (máxima similitud)
- **1** = Vectores opuestos (mínima similitud)

### Umbral de Relevancia

El parámetro `threshold` (0.4 por defecto) filtra resultados irrelevantes. Solo devuelve matches con distancia < threshold.

In [ ]:
def similarity_search(
    query_embedding: list[float],
    top_k: int = 3,
    group_filter: str = None,
    intent_filter: str = None,
    threshold: float = 0.4
) -> list[tuple]:
    """
    Busca los embeddings más similares a la consulta
    
    Parámetros:
    - query_embedding: Vector de la pregunta del usuario
    - top_k: Cantidad máxima de resultados
    - group_filter: Filtrar por grupo específico
    - intent_filter: Filtrar por intención específica
    - threshold: Umbral de distancia (0-1, menor = más estricto)
    
    Retorna: Lista de tuplas [(RAGEmbedding, distancia), ...]
    """
    session = get_db_session()
    
    try:
        # Calcular distancia coseno
        distance_expr = RAGEmbedding.embedding.cosine_distance(query_embedding)
        
        # Construir query
        query = session.query(
            RAGEmbedding,
            distance_expr.label("distance")
        )
        
        # Aplicar filtros
        if group_filter:
            query = query.filter(RAGEmbedding.group_name == group_filter)
        
        if intent_filter:
            query = query.filter(RAGEmbedding.intent == intent_filter)
        
        # CLAVE: Filtrar por umbral de relevancia
        query = query.filter(distance_expr < threshold)
        
        # Ordenar por similitud y limitar resultados
        results = query.order_by(distance_expr).limit(top_k).all()
        
        return results
        
    except Exception as e:
        print(f"Error en búsqueda: {e}")
        return []
    finally:
        session.close()

##  Ingesta de FAQs

El archivo `ingestor.py` procesa las FAQs desde un archivo JSON y las convierte en embeddings.

In [ ]:
import json
from langchain_ollama import OllamaEmbeddings

# Configurar el modelo de embeddings
embedder = OllamaEmbeddings(model="nomic-embed-text")

def ingest_faqs(path="files/faqs.json"):
    """
    Proceso de ingesta de FAQs:
    1. Leer archivo JSON con preguntas y respuestas
    2. Generar embeddings para cada pregunta
    3. Almacenar en la base de datos vectorial
    """
    
    # Cargar FAQs
    with open(path, "r", encoding="utf-8") as f:
        lista_faqs = json.load(f)
    
    print(f"Procesando {len(lista_faqs)} FAQs...")
    
    items_to_insert = []
    
    # Extraer solo las preguntas para vectorización
    preguntas = [item["pregunta"] for item in lista_faqs]
    
    # Generar embeddings en lote (más eficiente)
    print("Generando embeddings...")
    vectores = embedder.embed_documents(preguntas)
    
    # Preparar datos para inserción
    for i, item in enumerate(lista_faqs):
        faq_data = {
            # Texto completo: pregunta + respuesta
            "text": f"Pregunta: {item['pregunta']}\nRespuesta: {item['respuesta']}",
            "group_name": "faq_empresa",
            "intent": "faq_manual",
            "embedding": vectores[i],
            "meta": {"tipo": "faq_estatica"}
        }
        items_to_insert.append(faq_data)
    
    # Insertar en la BD
    add_batch_embeddings(items_to_insert)
    print(" FAQs cargadas exitosamente!")

# Ejecutar ingesta
# ingest_faqs()

##  Pruebas del Sistema

Ejemplo de cómo probar el sistema RAG con preguntas de usuarios:

In [ ]:
# Preguntas de prueba
preguntas_test = [
    "¿A qué hora abren la tienda?",
    "¿Tienen envíos a Guayaquil?",
    "Se me dañó el producto, ¿qué hago?",
    "¿Cuál es la mejor compu que tienen?",  # No está en FAQs
]

for pregunta in preguntas_test:
    print(f"\n Usuario: '{pregunta}'")
    
    # Generar embedding de la pregunta
    query_vector = embedder.embed_query(pregunta)
    
    # Buscar en la base de conocimiento
    resultados = similarity_search(
        query_embedding=query_vector,
        top_k=1,
        group_filter="faq_empresa",
        threshold=0.5  # Ajustar según necesidad
    )
    
    if not resultados:
        print(" No encontré información relevante en la base de conocimiento.")
    else:
        mejor_match, distancia = resultados[0]
        print(f" Distancia: {distancia:.4f}")
        print(f"📄 Respuesta: {mejor_match.text}")
    
    print("-" * 70)

##  Flujo de Trabajo Completo

```
1. INICIALIZACIÓN
   └─> create_vector_tables()
       └─> Crea extensión pgvector
       └─> Crea tabla rag_embeddings

2. INGESTA DE DATOS
   └─> ingest_faqs()
       └─> Lee faqs.json
       └─> Genera embeddings con Ollama
       └─> add_batch_embeddings()

3. CONSULTA DE USUARIO
   └─> Usuario hace pregunta
       └─> embedder.embed_query(pregunta)
       └─> similarity_search()
           └─> Calcula distancia coseno
           └─> Filtra por umbral
           └─> Retorna mejores matches
       └─> Retornar respuesta al usuario
```

##  Configuración y Optimización

### Parámetros Clave

| Parámetro | Valor | Descripción |
|-----------|-------|-------------|
| `model` | nomic-embed-text | Modelo de Ollama para embeddings |
| `vector_dim` | 768 | Dimensiones del vector |
| `threshold` | 0.4-0.5 | Umbral de relevancia |
| `top_k` | 1-3 | Cantidad de resultados |

### Ajuste del Threshold

- **0.3 o menos**: Muy estricto, solo matches casi exactos
- **0.4-0.5**: Balance entre precisión y cobertura (recomendado)
- **0.6 o más**: Permisivo, puede devolver resultados irrelevantes

### Mejores Prácticas

1. **Preparar bien el texto**: Incluir pregunta + respuesta en el campo `text`
2. **Usar filtros**: Aprovechar `group_name` e `intent` para búsquedas focalizadas
3. **Ajustar threshold**: Probar con datos reales y ajustar según resultados
4. **Indexar correctamente**: Usar índices HNSW para búsquedas rápidas
5. **Metadata útil**: Guardar información adicional en `meta` para contexto

##  Ventajas del Sistema RAG

 **Respuestas Precisas**: Basadas en información real de la empresa

 **Escalable**: Fácil agregar más FAQs sin reentrenar modelos

 **Rápido**: Búsqueda vectorial optimizada con pgvector

 **Controlable**: Umbral de relevancia evita respuestas incorrectas

 **Actualizable**: Cambios en FAQs se reflejan inmediatamente

##  Referencias

- **pgvector**: https://github.com/pgvector/pgvector
- **Ollama Embeddings**: https://ollama.ai/
- **LangChain**: https://python.langchain.com/
- **RAG Pattern**: https://arxiv.org/abs/2005.11401

---

*Documentación generada para ProyectoAprendizaje - Diciembre 2024*